In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import os

# === PATH SETUP ===
npy_path = Path(r"D:/adc/pokemon_with_4_moves.npy")
pokeapi_path = Path(r"D:/idm downloads/pokeapi-master/data/v2/csv")
output_csv = Path(r"D:/adc/team_battle_results.csv")

# === LOAD DATA ===
pokemon_moves_dict = np.load(npy_path, allow_pickle=True).item()
pokemon_df = pd.read_csv(pokeapi_path / "pokemon.csv")
moves_df = pd.read_csv(pokeapi_path / "moves.csv")
pokemon_stats_df = pd.read_csv(pokeapi_path / "pokemon_stats.csv")
stats_df = pd.read_csv(pokeapi_path / "stats.csv")

# === BUILD STAT MAPPINGS ===
stat_id_to_name = stats_df.set_index("id")["identifier"].to_dict()
pokemon_id_to_name = pokemon_df.set_index("id")["identifier"].to_dict()

# === BUILD POKEMON STATS DICTIONARY ===
poke_stats = {}
for pid, group in pokemon_stats_df.groupby("pokemon_id"):
    name = pokemon_id_to_name.get(pid)
    if not name:
        continue
    stats = {stat_id_to_name[row["stat_id"]]: row["base_stat"] for _, row in group.iterrows()}
    poke_stats[name] = stats

# === FILTER VALID POKEMON ===
valid_moves = set(moves_df["identifier"])
pokemon_moves_dict = {
    k: [m for m in v if m in valid_moves]
    for k, v in pokemon_moves_dict.items()
    if all(m in valid_moves for m in v)
}
valid_pokemon = list(set(pokemon_moves_dict) & set(poke_stats))

# === ROLE CLASSIFICATION ===
poke_roles = {}
for name in valid_pokemon:
    stats = poke_stats[name]
    hp, atk, def_, spatk, spdef, speed = stats["hp"], stats["attack"], stats["defense"], stats["special-attack"], stats["special-defense"], stats["speed"]
    total = hp + atk + def_ + spatk + spdef + speed
    if total < 400:
        continue
    if hp + def_ + spdef > 250:
        role = "Tank"
    elif speed > 80 and (atk > 80 or spatk > 80):
        role = "Sweeper"
    else:
        role = "Balanced"
    poke_roles[name] = role

# === GROUP BY ROLE ===
role_groups = defaultdict(list)
for name, role in poke_roles.items():
    role_groups[role].append(name)

# === STRATEGIC TEAM BUILDER ===
def generate_strategic_team(exclude=None):
    exclude = set(exclude or [])
    team = []
    used = set()

    def pick(role_list):
        available = [p for p in role_list if p not in used and p not in exclude]
        if not available:
            return random.choice([p for p in role_list if p not in exclude])
        chosen = random.choice(available)
        used.add(chosen)
        return chosen

    if len(role_groups["Tank"]) == 0 or len(role_groups["Sweeper"]) < 2 or len(role_groups["Balanced"]) < 3:
        return random.sample([p for p in valid_pokemon if p not in exclude], 6)

    team.append(pick(role_groups["Tank"]))
    team.append(pick(role_groups["Sweeper"]))
    team.append(pick(role_groups["Sweeper"]))
    team.extend(random.sample([p for p in role_groups["Balanced"] if p not in used and p not in exclude], 3))
    return team

# === BATTLE LOGIC ===
def simulate_team_battle(team1, team2):
    t1_hp = sum(poke_stats[p]["hp"] for p in team1)
    t2_hp = sum(poke_stats[p]["hp"] for p in team2)
    while t1_hp > 0 and t2_hp > 0:
        p1 = random.choice(team1)
        p2 = random.choice(team2)
        m1 = random.choice(pokemon_moves_dict[p1])
        m2 = random.choice(pokemon_moves_dict[p2])
        power1 = moves_df[moves_df["identifier"] == m1]["power"]
        power2 = moves_df[moves_df["identifier"] == m2]["power"]
        power1 = int(power1.values[0]) if not power1.empty and not pd.isna(power1.values[0]) else 50
        power2 = int(power2.values[0]) if not power2.empty and not pd.isna(power2.values[0]) else 50
        dmg1 = int(power1 * random.uniform(0.8, 1.2))
        dmg2 = int(power2 * random.uniform(0.8, 1.2))
        t2_hp -= dmg1
        t1_hp -= dmg2

    if t1_hp == t2_hp:
        return random.choice(["Team1", "Team2"])
    return "Team1" if t1_hp > t2_hp else "Team2"

# === SIMULATE AND SAVE ===
battle_data = []
print("⚔️ Simulating 100000 strategic 6v6 battles...")
for _ in tqdm(range(100000)):
    team1 = generate_strategic_team()
    team2 = generate_strategic_team(exclude=team1)
    winner = simulate_team_battle(team1, team2)
    battle_data.append({
        "team1": ",".join(team1),
        "team2": ",".join(team2),
        "winner": winner
    })

os.makedirs(output_csv.parent, exist_ok=True)
battle_df = pd.DataFrame(battle_data)
battle_df.to_csv(output_csv, index=False)
print(f"✅ Saved to {output_csv}")

# === OPTIONAL GYM INTEGRATION (you can use later) ===
# gym_df = pd.read_csv("D:/adc/PokemonGymLeaders.csv")
# Use gym_df['team'] to generate team2 while team1 is random


⚔️ Simulating 100000 strategic 6v6 battles...


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [08:48<00:00, 189.34it/s]


✅ Saved to D:\adc\team_battle_results.csv


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
import os
import pickle

# === PATH SETUP ===
os.makedirs("D:/adc", exist_ok=True)
output_csv = Path(r"D:/adc/team_battle_results.csv")
model_output = Path(r"D:/adc/team_predictor_model.pth")
encoder_output = Path(r"D:/adc/pokemon_encoder.pkl")

# === LOAD & SHUFFLE BATTLE RESULTS ===
battle_df = pd.read_csv(output_csv)
battle_df = battle_df.sample(frac=1, random_state=42).reset_index(drop=True)

# === ENCODE TEAMS ===
all_pokemon = sorted(list(set(pokemon for row in battle_df[['team1', 'team2']].values.flatten() for pokemon in row.split(','))))
pokemon_encoder = {name: idx for idx, name in enumerate(all_pokemon)}

def encode_team(team):
    vec = np.zeros(len(all_pokemon))
    for name in team.split(','):
        if name in pokemon_encoder:
            vec[pokemon_encoder[name]] = 1
    return vec

# === PREPARE FEATURES AND LABELS ===
X = []
y = []
for _, row in battle_df.iterrows():
    team1_vec = encode_team(row['team1'])
    team2_vec = encode_team(row['team2'])
    X.append(np.concatenate([team1_vec, team2_vec]))
    y.append(0 if row['winner'] == 'Team1' else 1)

X = np.array(X)
y = np.array(y)

# === SPLIT DATA ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === DATA LOADER ===
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# === MODEL DEFINITION ===
class TeamBattlePredictor(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.net(x)

model = TeamBattlePredictor(input_size=X.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === TRAINING LOOP ===
for epoch in range(20):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        output = model(xb)
        loss = criterion(output, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Evaluation
    model.eval()
    with torch.no_grad():
        pred_test = model(X_test_tensor)
        test_acc = (pred_test.argmax(dim=1) == y_test_tensor).float().mean().item()

        pred_train = model(X_train_tensor)
        train_acc = (pred_train.argmax(dim=1) == y_train_tensor).float().mean().item()

    print(f"Epoch {epoch+1}/20 - Loss: {total_loss:.4f} - Train Acc: {train_acc:.4f} - Test Acc: {test_acc:.4f}")

# === FINAL EVALUATION ===
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    print("\n📊 Classification Report:")
    print(classification_report(y_true, y_pred))
    print("✅ Final Accuracy:", accuracy_score(y_true, y_pred))

# === SAVE MODEL & ENCODER ===
torch.save(model.state_dict(), model_output)
with open(encoder_output, "wb") as f:
    pickle.dump(pokemon_encoder, f)

print(f"\n✅ Model saved to: {model_output}")
print(f"✅ Encoder saved to: {encoder_output}")


Epoch 1/20 - Loss: 725.7132 - Train Acc: 0.7397 - Test Acc: 0.7029
Epoch 2/20 - Loss: 660.6631 - Train Acc: 0.8013 - Test Acc: 0.7035
Epoch 3/20 - Loss: 521.5065 - Train Acc: 0.9499 - Test Acc: 0.6809
Epoch 4/20 - Loss: 181.3563 - Train Acc: 0.9896 - Test Acc: 0.6707
Epoch 5/20 - Loss: 45.3325 - Train Acc: 0.9960 - Test Acc: 0.6630
Epoch 6/20 - Loss: 18.0168 - Train Acc: 0.9974 - Test Acc: 0.6668
Epoch 7/20 - Loss: 17.0326 - Train Acc: 0.9961 - Test Acc: 0.6688
Epoch 8/20 - Loss: 25.5842 - Train Acc: 0.9951 - Test Acc: 0.6649
Epoch 9/20 - Loss: 21.5931 - Train Acc: 0.9972 - Test Acc: 0.6655
Epoch 10/20 - Loss: 14.1667 - Train Acc: 0.9978 - Test Acc: 0.6651
Epoch 11/20 - Loss: 12.7258 - Train Acc: 0.9973 - Test Acc: 0.6640
Epoch 12/20 - Loss: 11.9981 - Train Acc: 0.9978 - Test Acc: 0.6650
Epoch 13/20 - Loss: 10.9600 - Train Acc: 0.9981 - Test Acc: 0.6632
Epoch 14/20 - Loss: 9.8602 - Train Acc: 0.9984 - Test Acc: 0.6644
Epoch 15/20 - Loss: 9.1711 - Train Acc: 0.9989 - Test Acc: 0.6643
Ep

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path
import os
import random
from sklearn.model_selection import train_test_split
from difflib import get_close_matches

# === PATH SETUP ===
os.makedirs("D:/adc", exist_ok=True)
output_csv = Path(r"D:/adc/team_battle_results.csv")
model_output = Path(r"D:/adc/team_predictor_model.pth")
accuracy_log_path = Path("D:/adc/accuracy_log.csv")

# === LOAD BATTLE RESULTS ===
battle_df = pd.read_csv(output_csv).dropna(subset=["team1", "team2"])

# === ENCODE TEAMS ===
all_pokemon = sorted(list(set(pokemon for row in battle_df[['team1', 'team2']].values.flatten() for pokemon in row.split(','))))
pokemon_encoder = {name: idx for idx, name in enumerate(all_pokemon)}

def encode_team(team):
    vec = np.zeros(len(all_pokemon))
    for name in team.split(','):
        if name in pokemon_encoder:
            vec[pokemon_encoder[name]] = 1
    return vec

def prepare_training_data(df):
    X, y = [], []
    for _, row in df.iterrows():
        team1_vec = encode_team(row['team1'])
        team2_vec = encode_team(row['team2'])
        X.append(np.concatenate([team1_vec, team2_vec]))
        y.append(0 if row['winner'] == 'Team1' else 1)
    return np.array(X), np.array(y)

class TeamBattlePredictor(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )
    def forward(self, x):
        return self.net(x)

X, y = prepare_training_data(battle_df)
model = TeamBattlePredictor(input_size=X.shape[1])
model.load_state_dict(torch.load(model_output))
model.eval()

def predict_battle(team_user, team_ai):
    vec_user = encode_team(team_user)
    vec_ai = encode_team(team_ai)
    input_vec = np.concatenate([vec_user, vec_ai])
    input_tensor = torch.tensor(input_vec, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
        prob = torch.softmax(output, dim=1)[0].tolist()
        winner = "User" if output.argmax().item() == 0 else "AI"
    return winner, prob

def select_best_ai_team(user_team, all_pokemon, model, num_trials=5):
    best_team = None
    best_prob = -1
    for _ in range(num_trials):
        candidate_team = random.sample([p for p in all_pokemon if p not in user_team], 6)
        vec_user = encode_team(",".join(user_team))
        vec_ai = encode_team(",".join(candidate_team))
        input_vec = np.concatenate([vec_user, vec_ai])
        input_tensor = torch.tensor(input_vec, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            prob = torch.softmax(model(input_tensor), dim=1)[0, 1].item()
        if prob > best_prob:
            best_prob = prob
            best_team = candidate_team
    return best_team

def simulate_turns(user_team, ai_team):
    turns = 0
    user_health = 600
    ai_health = 600
    move_log = []
    while user_health > 0 and ai_health > 0 and turns < 20:
        turns += 1
        user_mon = random.choice(user_team)
        ai_mon = random.choice(ai_team)
        user_move = f"{user_mon} used Tackle"
        ai_move = f"{ai_mon} used Bite"
        dmg_user = random.randint(30, 60)
        dmg_ai = random.randint(30, 60)
        ai_health -= dmg_user
        user_health -= dmg_ai
        move_log.append((turns, user_move, dmg_user, ai_move, dmg_ai))
    if user_health == ai_health:
        winner = random.choice(["User", "AI"])
    else:
        winner = "User" if user_health > ai_health else "AI"
    print(f"📜 Battle Log - {turns} turns")
    for t, u_act, u_dmg, a_act, a_dmg in move_log:
        print(f"Turn {t}: {u_act} → {u_dmg} dmg | {a_act} → {a_dmg} dmg")
    print(f"Final HP → User: {max(user_health, 0)}, AI: {max(ai_health, 0)}")
    return winner

def log_accuracy(acc):
    with open(accuracy_log_path, "a") as f:
        f.write(f"{len(battle_df)},{acc:.4f}\n")

def retrain_model_on_new_data():
    print("🔄 Retraining model on new data...")
    df = pd.read_csv(output_csv)
    X, y = prepare_training_data(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for _ in range(5):
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        acc = (model(X_test_tensor).argmax(dim=1) == y_test_tensor).float().mean().item()
    log_accuracy(acc)
    torch.save(model.state_dict(), model_output)
    print(f"✅ Retrained model saved with test accuracy: {acc:.4f}")

# === MAIN ADAPTIVE BATTLE LOOP ===
num_trials = 2
while True:
    print("\n⚔️ Enter your team of 6 Pokémon (comma-separated) or 'q' to quit:")
    user_input = input("Team: ").strip()
    if user_input.lower() == 'q':
        print("👋 Quitting battle loop. Goodbye!")
        break

    raw_team = [p.strip() for p in user_input.split(',')]
    user_team = []
    for name in raw_team:
        if name in all_pokemon:
            user_team.append(name)
        else:
            match = get_close_matches(name, all_pokemon, n=1, cutoff=0.6)
            if match:
                print(f"🔁 Autocorrected '{name}' to '{match[0]}'")
                user_team.append(match[0])
            else:
                print(f"❌ '{name}' not recognized and has no close match.")
    if len(user_team) != 6:
        print("❌ Invalid team. Try again.")
        continue

    while True:
        ai_team = select_best_ai_team(user_team, all_pokemon, model, num_trials=num_trials)
        print(f"\n👾 AI BATTLE - Trials: {num_trials}")
        print("User Team:", user_team)
        print("AI Team:", ai_team)

        predicted_winner, prob = predict_battle(",".join(user_team), ",".join(ai_team))
        print(f"🤖 Model Prediction → Winner: {predicted_winner}")
        print(f"🔍 Confidence → User: {prob[0]:.2f}, AI: {prob[1]:.2f}")

        sim_winner = simulate_turns(user_team, ai_team)
        print(f"\n⚔️ Actual Battle Result → Winner: {sim_winner}")

        # === LOG BATTLE ===
        battle_df = pd.concat([battle_df, pd.DataFrame([{
            'team1': ",".join(user_team),
            'team2': ",".join(ai_team),
            'winner': 'Team1' if sim_winner == 'User' else 'Team2',
            'predicted': 'Team1' if predicted_winner == 'User' else 'Team2'
        }])], ignore_index=True)
        battle_df.to_csv(output_csv, index=False)

        if len(battle_df) % 20 == 0:
            retrain_model_on_new_data()

        if sim_winner == 'User':
            num_trials += 10
            print(f"\n🤖 You win! Want a rematch with stronger AI (tries: {num_trials})? (y/q):")
            response = input().strip().lower()
            if response == 'y':
                continue
            elif response == 'q':
                print("👋 Quitting battle loop. Goodbye!")
                exit()
            else:
                print("Invalid input. Returning to main menu.")
                break
        else:
            print(f"\n🤖 AI wins! Enter new team or 'q' to quit.")
            num_trials = 2
            break



⚔️ Enter your team of 6 Pokémon (comma-separated) or 'q' to quit:


Team:  darkrai,aggron,magnetron,dialga,palkiya,tortera


🔁 Autocorrected 'magnetron' to 'magneton'
🔁 Autocorrected 'palkiya' to 'palkia'
🔁 Autocorrected 'tortera' to 'torterra'

👾 AI BATTLE - Trials: 2
User Team: ['darkrai', 'aggron', 'magneton', 'dialga', 'palkia', 'torterra']
AI Team: ['feraligatr', 'eiscue-noice', 'braviary-hisui', 'delcatty', 'ursaluna-bloodmoon', 'runerigus']
🤖 Model Prediction → Winner: User
🔍 Confidence → User: 0.82, AI: 0.18
📜 Battle Log - 14 turns
Turn 1: aggron used Tackle → 44 dmg | braviary-hisui used Bite → 37 dmg
Turn 2: aggron used Tackle → 39 dmg | feraligatr used Bite → 31 dmg
Turn 3: torterra used Tackle → 34 dmg | ursaluna-bloodmoon used Bite → 51 dmg
Turn 4: magneton used Tackle → 48 dmg | braviary-hisui used Bite → 48 dmg
Turn 5: palkia used Tackle → 60 dmg | eiscue-noice used Bite → 37 dmg
Turn 6: magneton used Tackle → 60 dmg | ursaluna-bloodmoon used Bite → 31 dmg
Turn 7: palkia used Tackle → 48 dmg | eiscue-noice used Bite → 53 dmg
Turn 8: palkia used Tackle → 30 dmg | delcatty used Bite → 45 dmg
Tur

 y



👾 AI BATTLE - Trials: 12
User Team: ['darkrai', 'aggron', 'magneton', 'dialga', 'palkia', 'torterra']
AI Team: ['chansey', 'pelipper', 'heatran', 'ninjask', 'koraidon', 'swellow']
🤖 Model Prediction → Winner: AI
🔍 Confidence → User: 0.00, AI: 1.00
📜 Battle Log - 14 turns
Turn 1: aggron used Tackle → 54 dmg | ninjask used Bite → 30 dmg
Turn 2: darkrai used Tackle → 58 dmg | koraidon used Bite → 45 dmg
Turn 3: magneton used Tackle → 44 dmg | koraidon used Bite → 34 dmg
Turn 4: torterra used Tackle → 37 dmg | swellow used Bite → 32 dmg
Turn 5: palkia used Tackle → 31 dmg | koraidon used Bite → 44 dmg
Turn 6: palkia used Tackle → 56 dmg | ninjask used Bite → 30 dmg
Turn 7: magneton used Tackle → 48 dmg | koraidon used Bite → 40 dmg
Turn 8: aggron used Tackle → 52 dmg | heatran used Bite → 59 dmg
Turn 9: magneton used Tackle → 41 dmg | heatran used Bite → 44 dmg
Turn 10: dialga used Tackle → 51 dmg | chansey used Bite → 41 dmg
Turn 11: torterra used Tackle → 37 dmg | pelipper used Bite → 5

...

🔁 Autocorrected 'magnetron' to 'magneton'
🔁 Autocorrected 'palkiya' to 'palkia'
🔁 Autocorrected 'tortera' to 'torterra'

👾 AI BATTLE - Trials: 2
User Team: ['darkrai', 'aggron', 'magneton', 'dialga', 'palkia', 'torterra']
AI Team: ['swampert', 'galvantula', 'tornadus-incarnate', 'goodra-hisui', 'ampharos-mega', 'zygarde-10']
🤖 Model Prediction → Winner: User
🔍 Confidence → User: 0.98, AI: 0.02
📜 Battle Log - 13 turns
Turn 1: darkrai used Tackle → 51 dmg | tornadus-incarnate used Bite → 43 dmg
Turn 2: torterra used Tackle → 50 dmg | galvantula used Bite → 57 dmg
Turn 3: dialga used Tackle → 40 dmg | ampharos-mega used Bite → 55 dmg
Turn 4: magneton used Tackle → 34 dmg | goodra-hisui used Bite → 47 dmg
Turn 5: palkia used Tackle → 53 dmg | ampharos-mega used Bite → 36 dmg
Turn 6: aggron used Tackle → 55 dmg | ampharos-mega used Bite → 59 dmg
Turn 7: dialga used Tackle → 57 dmg | ampharos-mega used Bite → 40 dmg
Turn 8: palkia used Tackle → 53 dmg | tornadus-incarnate used Bite → 37 dmg


 y



👾 AI BATTLE - Trials: 12
User Team: ['darkrai', 'aggron', 'magneton', 'dialga', 'palkia', 'torterra']
AI Team: ['kangaskhan', 'hydrapple', 'hawlucha', 'mantine', 'archen', 'honchkrow']
🤖 Model Prediction → Winner: AI
🔍 Confidence → User: 0.00, AI: 1.00
📜 Battle Log - 14 turns
Turn 1: aggron used Tackle → 45 dmg | honchkrow used Bite → 31 dmg
Turn 2: torterra used Tackle → 33 dmg | hawlucha used Bite → 31 dmg
Turn 3: aggron used Tackle → 60 dmg | hawlucha used Bite → 40 dmg
Turn 4: darkrai used Tackle → 45 dmg | hydrapple used Bite → 55 dmg
Turn 5: dialga used Tackle → 47 dmg | honchkrow used Bite → 51 dmg
Turn 6: darkrai used Tackle → 42 dmg | hydrapple used Bite → 51 dmg
Turn 7: dialga used Tackle → 59 dmg | mantine used Bite → 33 dmg
Turn 8: palkia used Tackle → 41 dmg | kangaskhan used Bite → 59 dmg
Turn 9: aggron used Tackle → 56 dmg | kangaskhan used Bite → 43 dmg
Turn 10: darkrai used Tackle → 37 dmg | hydrapple used Bite → 46 dmg
Turn 11: palkia used Tackle → 55 dmg | hydrapple

 y



👾 AI BATTLE - Trials: 22
User Team: ['darkrai', 'aggron', 'magneton', 'dialga', 'palkia', 'torterra']
AI Team: ['slowbro-galar', 'chi-yu', 'metagross', 'darmanitan-galar-standard', 'quaquaval', 'eternatus-eternamax']
🤖 Model Prediction → Winner: AI
🔍 Confidence → User: 0.00, AI: 1.00
📜 Battle Log - 14 turns
Turn 1: aggron used Tackle → 54 dmg | chi-yu used Bite → 43 dmg
Turn 2: aggron used Tackle → 43 dmg | quaquaval used Bite → 58 dmg
Turn 3: torterra used Tackle → 34 dmg | quaquaval used Bite → 37 dmg
Turn 4: dialga used Tackle → 36 dmg | metagross used Bite → 48 dmg
Turn 5: torterra used Tackle → 54 dmg | chi-yu used Bite → 31 dmg
Turn 6: palkia used Tackle → 30 dmg | slowbro-galar used Bite → 34 dmg
Turn 7: aggron used Tackle → 47 dmg | quaquaval used Bite → 51 dmg
Turn 8: magneton used Tackle → 47 dmg | quaquaval used Bite → 52 dmg
Turn 9: torterra used Tackle → 43 dmg | chi-yu used Bite → 48 dmg
Turn 10: magneton used Tackle → 42 dmg | slowbro-galar used Bite → 44 dmg
Turn 11: m

In [43]:
def find_user_team_to_defeat_ai(model, all_pokemon):
    def encode_team(team):
        vec = np.zeros(len(all_pokemon))
        for name in team.split(','):
            if name in pokemon_encoder:
                vec[pokemon_encoder[name]] = 1
        return vec

    best_user_team = None
    best_ai_team = None
    lowest_ai_win_prob = 1.0

    for _ in range(200):
        user_team = random.sample(all_pokemon, 6)

        # 🔁 Let AI generate its best counter-team against this user team
        ai_team = select_best_ai_team(user_team, all_pokemon, model)

        vec_user = encode_team(",".join(user_team))
        vec_ai = encode_team(",".join(ai_team))
        input_vec = np.concatenate([vec_user, vec_ai])
        input_tensor = torch.tensor(input_vec, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            ai_win_prob = torch.softmax(model(input_tensor), dim=1)[0, 1].item()

        if ai_win_prob < lowest_ai_win_prob:
            lowest_ai_win_prob = ai_win_prob
            best_user_team = user_team
            best_ai_team = ai_team

    print("\n🛡️ Best User Team Found to Defeat AI (based on AI’s own response)")
    print("User Team:", best_user_team)
    print("AI Team:", best_ai_team)
    print(f"Win Probability (User): {1 - lowest_ai_win_prob:.2f}")
    return best_user_team, best_ai_team

# Run it:
user_team, ai_team = find_user_team_to_defeat_ai(model, all_pokemon)



🛡️ Best User Team Found to Defeat AI (based on AI’s own response)
User Team: ['suicune', 'sliggoo', 'tatsugiri-curly', 'clodsire', 'sandy-shocks', 'lickilicky']
AI Team: ['victini', 'cofagrigus', 'ludicolo', 'lycanroc-midnight', 'skuntank', 'venomoth']
Win Probability (User): 1.00
